# The Template Pattern - Forcing a Structured Response

Hello everyone. Whenever interacting with LLMs, it will very often be the case that the model's reasoning and its final answer are mixed together in a free-form block of text, making it difficult for our programs to use the output reliably.

This is where a highly practical and robust technique comes in: **The Template Pattern.**

The core idea is profound: instead of letting the model decide the structure of its response, **we force the model to fill in a predefined template.** This gives us complete control over the entire output, making it perfectly and predictably parsable.

This notebook will demonstrate how to use this pattern to separate the model's "thinking" from its final "answer."

## The Task and Helper Functions

Our task will be to refactor a piece of Python code. We want the model to first *think* about the necessary changes and then provide the final code. We will compare a standard Chain-of-Thought response with a templated one.

In [ ]:
import inspect
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500

# --- The "messy" code to be refactored ---
def get_user_initials(user_data_list):
    initials_list = []
    for user in user_data_list:
        if 'name' in user and len(user['name']) > 0:
            parts = user['name'].split(' ')
            initials = ''
            for part in parts:
                initials += part[0].upper()
            initials_list.append(initials)
    return initials_list

CODE_TO_REFACTOR = inspect.getsource(get_user_initials)

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

print("Setup complete. Helper functions and code context are ready.")

## Unstructured Chain-of-Thought

Let's start by using a standard Chain-of-Thought prompt. We'll ask the model to think step-by-step and then provide the code. Watch how the reasoning and the final code are intertwined in the output.

## The Template Pattern

Now, we'll solve this problem by providing a strict template. We will define clear sections using custom tags like `[THOUGHTS]` and `[REFACTORED_CODE]` and instruct the model to place its response *only* within these sections.